In [1]:
import sys
# append the path of the parent directory
sys.path.append("..")

In [2]:
import math
import os

import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
import pandas as pd
import datetime

import random
random.seed(7)

import orjson
import gzip

from importlib import reload
from lib import sketches, visualization_utils, encoders, ploting, pacha_sketch

reload(pacha_sketch)
from lib.pacha_sketch import ADTree

/home/rudi/.local/lib/python3.12/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "
/usr/lib/python3/dist-packages/pytz/__init__.py:31: SyntaxWarning: invalid escape sequence '\s'
  match = re.match("^#\s*version\s*([0-9a-z]*)\s*$", line)


# Helper Functions

In [3]:
def convert_np_types(obj):
    if isinstance(obj, dict):
        return {k: convert_np_types(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_np_types(v) for v in obj]
    elif isinstance(obj, (np.integer, np.floating)):
        return obj.item()
    else:
        return obj

In [4]:
def generate_queries(df: pd.DataFrame, num_cols: list, cat_cols: list, num_queries: int = 200, n_cat: int = 1, n_num: int = 1,
                      decay_rate: float = 0.5, range_portion:float = 0.5, dataset_name: str = None, file_path: str = None) -> list:
    # Exponentially decaying probability for each index - smaller decay_rate means faster decay
    probs_cat = np.array([decay_rate**i for i in range(len(cat_cols))])
    probs_cat = probs_cat / probs_cat.sum()

    probs_num = np.array([decay_rate**i for i in range(len(num_cols))])
    probs_num = probs_num / probs_num.sum()

    queries = []
    for i in range(num_queries):
        filter_predicates = {}

        picked_cat_cols = np.random.choice(cat_cols, size=n_cat, replace=False, p=probs_cat)
        for col in picked_cat_cols:
            val_counts = df[col].value_counts(normalize=True)
            predicate = list(np.random.choice(val_counts.index, size=1, p=val_counts.values))
            filter_predicates[col] = predicate

        picked_num_cols = np.random.choice(num_cols, size=n_num, replace=False, p=probs_num)
        for col in picked_num_cols:
            q_25 = int(df[col].quantile(0.25))
            q_75 = int(df[col].quantile(0.75))
            range_size = int((q_75 - q_25) * range_portion)
            start = random.randint(q_25, q_75 - range_size)
            end = start + range_size
            filter_predicates[col] = (start, end)

        all_predicates = []
        for col in df.columns:
            if col in filter_predicates:
                all_predicates.append(filter_predicates[col])
            else:
                all_predicates.append("*")
        queries.append(all_predicates)
    
    query_set = {
        "dataset_name": dataset_name,
        "n_cat": n_cat,
        "n_num": n_num,
        "num_queries": num_queries,
        "range_size": range_size,
        "queries": queries
    }

    if file_path is not None:
        cleaned_query_set = convert_np_types(query_set)
        if file_path.endswith('.gz'):
            with gzip.open(file_path, "wb") as f:
                f.write(orjson.dumps(cleaned_query_set))
        elif file_path.endswith('.json'):
            with open(file_path, 'wb') as f:
                f.write(orjson.dumps(cleaned_query_set))
                
    return query_set


In [5]:
def profile_num_cols(df: pd.DataFrame, num_cols: list[str], bases: list[int] = None, range_portion: float = 0.5):
    if bases is None:
        bases = [2] * len(num_cols)
    ranges_df = (df[num_cols].quantile(0.75) - df[num_cols].quantile(0.25)) * range_portion
    print("Range sizes for numerical columns:")
    print(ranges_df)
    range_levels = np.log(ranges_df) / np.log(bases)
    print("\nApprox level for every range query:")
    print(range_levels)
    max_levels = np.log(df[num_cols].max()) / np.log(bases)
    print("\nMax levels:")
    print(max_levels)
    

In [6]:
def generate_ad_tree(df: pd.DataFrame, cat_cols: list[str], file_path:str = None) -> ADTree:
    """
    Generate an ADTree from the given DataFrame.
    
    :param df: DataFrame containing the data.
    :param cat_cols: List of categorical columns to be used in the ADTree.
    :param file_path: Optional path to save the ADTree.
    :return: An instance of ADTree.
    """
    ad_tree = ADTree()

    for col in cat_cols:
        ad_tree.add_dimension(set(df[col].unique().tolist()), name=col)

    print(ad_tree.names)
    
    if file_path is not None:
        ad_tree.save_to_file(file_path)
    return ad_tree

In [7]:
def partition_and_save(df: pd.DataFrame, n_partitions: int, base_path: str):
    dir_path = os.path.dirname(base_path)
    if dir_path and not os.path.exists(dir_path):
        os.makedirs(dir_path, exist_ok=True)
    partition_size = len(df) // n_partitions
    for i in range(n_partitions):
        start_idx = i * partition_size
        end_idx = (i + 1) * partition_size if i < n_partitions - 1 else len(df)
        partition = df.iloc[start_idx:end_idx]
        partition.to_parquet(f"{base_path}_part_{i+1}.parquet", index=False)

# Bank Marketing

https://archive.ics.uci.edu/dataset/222/bank+marketing

## Data Cleaning

In [ ]:
df = pd.read_parquet("raw/bank_marketing/bank_full.parquet")

In [ ]:
# Convert month and day to day of the year indexed from 0
df['date'] = df.apply(lambda row: (datetime.datetime.strptime(f"{row['month']} {row['day']}", "%b %d").timetuple().tm_yday - 1), axis=1)

In [ ]:
clean_df = df[['poutcome', 'job', 'education', 'housing', 'loan', 'marital', 'duration', 'balance', 'age', 'date']]

In [32]:
clean_df.to_parquet("clean/bank_marketing.parquet", index=False)

## Partition Dataset

In [ ]:
bank_df = pd.read_parquet("clean/bank_marketing.parquet")

partition_and_save(bank_df, 10, "partitioned/bank_marketing_10/bank_marketing")

## Queries and AD-Tree Generation

In [5]:
bank_df = pd.read_parquet("clean/bank_marketing.parquet")
cat_cols = ['poutcome', 'job', 'education', 'housing', 'loan', 'marital']
num_cols = ['duration', 'balance', 'age', 'date']

In [34]:
bank_df.nunique()

poutcome        4
job            12
education       4
housing         2
loan            2
marital         3
duration     1573
balance      7168
age            77
date          318
dtype: int64

In [6]:
bank_df[num_cols].describe()

,duration,balance,age,date
count,45211.000000,45211.000000,45211.000000,45211.000000
mean,258.163080,1362.272058,40.936210,170.198668
std,257.527812,3044.765829,10.618762,74.597784
min,0.000000,-8019.000000,18.000000,5.000000
25%,103.000000,72.000000,33.000000,128.000000
50%,180.000000,448.000000,39.000000,154.000000
75%,319.000000,1428.000000,48.000000,216.000000
max,4918.000000,102127.000000,95.000000,364.000000


In [12]:
maxes = bank_df[num_cols].max().to_numpy()
mins = bank_df[num_cols].min().to_numpy()

maxes[1] = 803
mins[1] = 125
space_size = np.prod(np.abs(maxes - mins))

In [21]:
2**np.arange(5) 

array([ 1,  2,  4,  8, 16])

In [19]:
6**4

1296

In [13]:
space_size

np.int64(92172929772)

In [15]:
space_size / 154_789_372

np.float64(595.4732458763383)

In [38]:
profile_num_cols(bank_df, num_cols, bases=[4, 5, 2, 2])

Range sizes for numerical columns:
duration    108.0
balance     678.0
age           7.5
date         44.0
dtype: float64

Approx level for every range query:
duration    3.377444
balance     4.050574
age         2.906891
date        5.459432
dtype: float64

Max levels:
duration    6.131928
balance     7.166460
age         6.569856
date        8.507795
dtype: float64


In [ ]:
ad_tree = ADTree()

for col in cat_cols:
    ad_tree.add_dimension(set(bank_df[col].unique().tolist()), name=col)

ad_tree.save_to_file("sketches/ad_trees/bank_marketing.json")
print(ad_tree.names)

In [10]:
generate_queries(df=bank_df, num_cols=num_cols, cat_cols=cat_cols, num_queries=200, n_cat=1, n_num=1, dataset_name="bank_marketing_2_cols", file_path="../queries/bank_marketing_2_cols.json")

{'dataset_name': 'bank_marketing_2_cols',
 'n_cat': 1,
 'n_num': 1,
 'num_queries': 200,
 'range_size': 7,
 'queries': [[['unknown'], '*', '*', '*', '*', '*', '*', (125, 803), '*', '*'],
  [['unknown'], '*', '*', '*', '*', '*', (116, 224), '*', '*', '*'],
  ['*', '*', ['secondary'], '*', '*', '*', (103, 211), '*', '*', '*'],
  ['*', '*', '*', ['no'], '*', '*', '*', (652, 1330), '*', '*'],
  [['unknown'], '*', '*', '*', '*', '*', '*', (226, 904), '*', '*'],
  ['*', '*', ['secondary'], '*', '*', '*', '*', '*', (41, 48), '*'],
  ['*', '*', ['secondary'], '*', '*', '*', '*', '*', '*', (134, 178)],
  ['*', '*', ['tertiary'], '*', '*', '*', (149, 257), '*', '*', '*'],
  [['failure'], '*', '*', '*', '*', '*', '*', (700, 1378), '*', '*'],
  ['*', ['technician'], '*', '*', '*', '*', (106, 214), '*', '*', '*'],
  [['unknown'], '*', '*', '*', '*', '*', '*', '*', (34, 41), '*'],
  [['unknown'], '*', '*', '*', '*', '*', (129, 237), '*', '*', '*'],
  ['*', ['retired'], '*', '*', '*', '*', '*', '*', 

In [12]:
generate_queries(df=bank_df, num_cols=num_cols, cat_cols=cat_cols, num_queries=200, n_cat=2, n_num=2, dataset_name="bank_marketing_4_cols", file_path="../queries/bank_marketing_4_cols.json")

{'dataset_name': 'bank_marketing_4_cols',
 'n_cat': 2,
 'n_num': 2,
 'num_queries': 200,
 'range_size': 678,
 'queries': [[['unknown'],
   ['admin.'],
   '*',
   '*',
   '*',
   '*',
   (164, 272),
   (640, 1318),
   '*',
   '*'],
  ['*',
   ['management'],
   '*',
   '*',
   ['no'],
   '*',
   (174, 282),
   (180, 858),
   '*',
   '*'],
  ['*',
   ['admin.'],
   ['secondary'],
   '*',
   '*',
   '*',
   '*',
   '*',
   (33, 40),
   (143, 187)],
  [['unknown'],
   '*',
   ['secondary'],
   '*',
   '*',
   '*',
   (127, 235),
   (355, 1033),
   '*',
   '*'],
  [['unknown'],
   '*',
   ['secondary'],
   '*',
   '*',
   '*',
   (108, 216),
   '*',
   (34, 41),
   '*'],
  [['unknown'],
   '*',
   ['secondary'],
   '*',
   '*',
   '*',
   (160, 268),
   '*',
   (41, 48),
   '*'],
  [['unknown'],
   ['technician'],
   '*',
   '*',
   '*',
   '*',
   (106, 214),
   '*',
   (41, 48),
   '*'],
  ['*',
   ['blue-collar'],
   '*',
   ['yes'],
   '*',
   '*',
   (200, 308),
   '*',
   (34, 41),
  

# Online Retail

https://archive.ics.uci.edu/dataset/352/online+retail

In [15]:
df = pd.read_parquet("raw/online_retail/online_retail.parquet")

In [ ]:
df['category'] = df['StockCode'].str.slice(0, 3)
df['date'] = pd.to_datetime(df['InvoiceDate'], format="%d.%m.%Y %H:%M").dt.strftime('%d.%m.%Y').rank(method='dense').astype(int) - 1

In [ ]:
df_customer = pd.DataFrame(df['CustomerID'].unique(), columns=['CustomerID'])
df_customer['age'] = generate_bounded_normal(loc=35, scale=10, size=len(df_customer), low=18, high=76)
df_customer['gender'] = np.random.choice(['m', 'f', 'd'], size=len(df_customer), replace=True, p=[0.48, 0.49, 0.03])
df_merged = df.merge(df_customer, on='CustomerID', how='inner')
df_merged["region"] = df_merged['Country']
df_merged['total'] = df_merged['Quantity'] * df_merged['UnitPrice']
df_merged['total'] = df_merged['total'].abs().fillna(0).astype(int)

In [ ]:
clean_df = df_merged[['category', 'region', 'gender', 'date', 'age', 'total']].copy()

In [ ]:
clean_df.to_parquet("clean/online_retail.parquet", index=False)

## Partition Dataset

In [ ]:
retail_df = pd.read_parquet("clean/online_retail.parquet")

partition_and_save(retail_df, 10, "partitioned/online_retail_10/online_retail")

## Queries and AD-Tree Generation

In [16]:
retail_df = pd.read_parquet("clean/online_retail.parquet")
cat_cols = ['category', 'region', 'gender']
num_cols = ['date', 'age', 'total']

In [17]:
len(retail_df)

541909

In [23]:
retail_df["total"].quantile(0.99)

np.float64(192.0)

In [28]:
# Remove rows with outliers in 'total' using the IQR method
# Q1 = retail_df["total"].quantile(0.25)
# Q3 = retail_df["total"].quantile(0.75)
# IQR = Q3 - Q1
# lower_bound = Q1 - 1.5 * IQR
upper_bound = retail_df["total"].quantile(0.99)

retail_df_no = retail_df[(retail_df["total"] <= upper_bound)]
retail_df_no.to_parquet("clean/online_retail_no_outliers.parquet", index=False)


In [34]:
retail_df.nunique()

category     125
region        38
gender         3
total       1088
age           51
date         305
dtype: int64

In [61]:
retail_df[num_cols].describe()

,date,age,total
count,541909.000000,541909.000000,541909.000000
mean,151.535738,35.104582,20.881216
std,88.132067,7.753374,378.638866
min,0.000000,18.000000,0.000000
25%,74.000000,31.000000,3.000000
50%,150.000000,34.000000,9.000000
75%,228.000000,39.000000,17.000000
max,304.000000,71.000000,168469.000000


In [8]:
profile_num_cols(retail_df, num_cols, bases=[2, 2, 5])

Range sizes for numerical columns:
date     77.0
age       4.0
total     7.0
dtype: float64

Approx level for every range query:
date     6.266787
age      2.000000
total    1.209062
dtype: float64

Max levels:
date     8.247928
age      6.149747
total    7.477460
dtype: float64


In [51]:
generate_ad_tree(df=retail_df, cat_cols=cat_cols, file_path="sketches/ad_trees/online_retail.json")

['category', 'region', 'gender']


In [ ]:
generate_queries(df=retail_df, num_cols=num_cols, cat_cols=cat_cols, num_queries=200, n_cat=1, n_num=1, dataset_name="online_retail_2_cols", file_path="queries/online_retail_2_cols.json")

In [11]:
with open("../queries/online_retail_2_cols.json", 'wb') as f:
    f.write(orjson.dumps(retail_queries_2))

In [86]:
generate_queries(df=retail_df, num_cols=num_cols, cat_cols=cat_cols, num_queries=200, n_cat=2, n_num=2, dataset_name="online_retail_4_cols", file_path="queries/online_retail_4_cols.json")

{'dataset_name': 'online_retail_4_cols',
 'n_cat': 2,
 'n_num': 2,
 'num_queries': 200,
 'range_size': 7,
 'queries': [[['235'], ['United Kingdom'], '*', (140, 217), (35, 39), '*'],
  [['226'], ['Germany'], '*', (147, 224), (34, 38), '*'],
  [['229'], ['United Kingdom'], '*', (102, 179), '*', (5, 12)],
  [['229'], ['United Kingdom'], '*', (141, 218), (31, 35), '*'],
  [['214'], ['United Kingdom'], '*', (141, 218), (33, 37), '*'],
  [['228'], '*', ['f'], '*', (35, 39), (6, 13)],
  [['216'], '*', ['m'], (120, 197), (32, 36), '*'],
  [['228'], ['United Kingdom'], '*', (96, 173), (32, 36), '*'],
  [['850'], ['United Kingdom'], '*', (132, 209), '*', (5, 12)],
  [['226'], ['United Kingdom'], '*', (79, 156), (32, 36), '*'],
  [['233'], ['United Kingdom'], '*', (122, 199), '*', (8, 15)],
  [['849'], '*', ['f'], (120, 197), '*', (9, 16)],
  [['224'], ['United Kingdom'], '*', (89, 166), (34, 38), '*'],
  [['212'], ['United Kingdom'], '*', (93, 170), '*', (7, 14)],
  [['220'], '*', ['m'], (122, 1

# Folktables

https://github.com/socialfoundations/folktables

| Column  | Description                                                                       |
| ------- | --------------------------------------------------------------------------------- |
| `SEX`   | **Sex** — 1 for male, 2 for female.                                               |
| `RAC1P` | **Race** — Detailed race code (White, Black, Asian, etc.).                        |
| `SCHL`  | **Educational attainment** — Highest degree or level of school completed.         |
| `MAR`   | **Marital status** — E.g., married, divorced, widowed, never married.             |
| `POBP`  | **Place of birth** — Numeric code for U.S. state or foreign country of birth.     |
| `COW`   | **Class of worker** — Employment type (e.g., private, government, self-employed). |
| `OCCP`  | **Occupation code** — Detailed job classification (4-digit code).                 |
| `AGEP`  | **Age of person** — Age in years (0–99, top-coded at 99).                         |
| `PINCP` | **Total person income** — Total pre-tax income in the past 12 months.             |
| `PWGTP` | **Person’s weight** — Statistical weight used to produce population estimates.    |



In [ ]:
from folktables import ACSDataSource, ACSIncome, generate_categories, adult_filter

In [ ]:
data_source = ACSDataSource(survey_year='2017', horizon='1-Year', survey='person')
ca_data = data_source.get_data(states=["CA"], download=False)


In [54]:
cat_cols = ["SEX", "RAC1P", "SCHL", "MAR", "POBP", "COW", "OCCP"]
num_cols = ["AGEP", "PINCP", "PWGTP"]

In [ ]:
folk_df = ca_data[cat_cols + num_cols]
folk_df.fillna(0, inplace=True)
folk_df = folk_df.astype(int)

In [ ]:
folk_df.to_parquet("clean/acs_folktables.parquet", index=False)

## Partition Dataset

In [ ]:
folk_df = pd.read_parquet("clean/acs_folktables.parquet")

partition_and_save(folk_df, 10, "partitioned/acs_folktables_10/acs_folktables")

## Queries and AD-Tree Generation

In [ ]:
folk_df = pd.read_parquet("clean/acs_folktables.parquet")
cat_cols = ["SEX", "RAC1P", "SCHL", "MAR", "POBP", "COW", "OCCP"]
num_cols = ["AGEP", "PINCP", "PWGTP"]

In [88]:
folk_df.nunique()

SEX         2
RAC1P       9
SCHL       25
MAR         5
POBP      223
COW        10
OCCP      480
AGEP       92
PINCP    7877
PWGTP     811
dtype: int64

In [89]:
folk_df[num_cols].describe()

,AGEP,PINCP,PWGTP
count,377575.000000,3.775750e+05,377575.000000
mean,39.979792,3.736253e+04,104.712052
std,23.141297,6.580092e+04,75.371747
min,0.000000,-7.000000e+03,1.000000
25%,20.000000,0.000000e+00,59.000000
50%,39.000000,1.500000e+04,81.000000
75%,58.000000,4.800000e+04,126.000000
max,94.000000,1.259000e+06,1494.000000


In [7]:
profile_num_cols(folk_df, num_cols, bases=[2, 5, 2])

Range sizes for numerical columns:
AGEP        19.0
PINCP    24000.0
PWGTP       33.5
dtype: float64

Approx level for every range query:
AGEP     4.247928
PINCP    6.266666
PWGTP    5.066089
dtype: float64

Max levels:
AGEP      6.554589
PINCP     8.727164
PWGTP    10.544964
dtype: float64


In [92]:
generate_ad_tree(df=folk_df, cat_cols=cat_cols, file_path="sketches/ad_trees/acs_folktables.json")

['SEX', 'RAC1P', 'SCHL', 'MAR', 'POBP', 'COW', 'OCCP']


In [8]:
generate_queries(df=folk_df, num_cols=num_cols, cat_cols=cat_cols, num_queries=200, n_cat=1, n_num=1, dataset_name="acs_folktables_2_cols", file_path="queries/acs_folktables_2_cols.json")

{'dataset_name': 'acs_folktables_2_cols',
 'n_cat': 1,
 'n_num': 1,
 'num_queries': 200,
 'range_size': 19,
 'queries': [[[np.int64(2)], '*', '*', '*', '*', '*', '*', (30, 49), '*', '*'],
  [[np.int64(1)], '*', '*', '*', '*', '*', '*', (24, 43), '*', '*'],
  ['*', '*', '*', '*', [np.int64(6)], '*', '*', (32, 51), '*', '*'],
  [[np.int64(1)], '*', '*', '*', '*', '*', '*', '*', '*', (62, 95)],
  [[np.int64(2)], '*', '*', '*', '*', '*', '*', (22, 41), '*', '*'],
  ['*', [np.int64(1)], '*', '*', '*', '*', '*', (37, 56), '*', '*'],
  [[np.int64(1)], '*', '*', '*', '*', '*', '*', '*', '*', (65, 98)],
  [[np.int64(1)], '*', '*', '*', '*', '*', '*', (31, 50), '*', '*'],
  ['*', [np.int64(2)], '*', '*', '*', '*', '*', '*', (19096, 43096), '*'],
  [[np.int64(1)], '*', '*', '*', '*', '*', '*', (21, 40), '*', '*'],
  ['*', [np.int64(1)], '*', '*', '*', '*', '*', '*', (16627, 40627), '*'],
  [[np.int64(2)], '*', '*', '*', '*', '*', '*', '*', (7035, 31035), '*'],
  ['*', [np.int64(1)], '*', '*', '*'

In [9]:
generate_queries(df=folk_df, num_cols=num_cols, cat_cols=cat_cols, num_queries=200, n_cat=2, n_num=2, dataset_name="acs_folktables_4_cols", file_path="queries/acs_folktables_4_cols.json")

{'dataset_name': 'acs_folktables_4_cols',
 'n_cat': 2,
 'n_num': 2,
 'num_queries': 200,
 'range_size': 19,
 'queries': [[[np.int64(2)],
   [np.int64(1)],
   '*',
   '*',
   '*',
   '*',
   '*',
   (32, 51),
   '*',
   (68, 101)],
  [[np.int64(2)],
   [np.int64(1)],
   '*',
   '*',
   '*',
   '*',
   '*',
   '*',
   (11383, 35383),
   (75, 108)],
  [[np.int64(1)],
   [np.int64(8)],
   '*',
   '*',
   '*',
   '*',
   '*',
   (31, 50),
   (19735, 43735),
   '*'],
  ['*',
   [np.int64(9)],
   [np.int64(0)],
   '*',
   '*',
   '*',
   '*',
   (23, 42),
   (15536, 39536),
   '*'],
  ['*',
   '*',
   [np.int64(16)],
   '*',
   '*',
   '*',
   [np.int64(9130)],
   (35, 54),
   (3779, 27779),
   '*'],
  [[np.int64(1)],
   '*',
   [np.int64(22)],
   '*',
   '*',
   '*',
   '*',
   (34, 53),
   '*',
   (89, 122)],
  [[np.int64(2)],
   [np.int64(1)],
   '*',
   '*',
   '*',
   '*',
   '*',
   (35, 54),
   (10218, 34218),
   '*'],
  [[np.int64(2)],
   [np.int64(3)],
   '*',
   '*',
   '*',
   '*',

# OLD

### Self-Generated Datasets

In [ ]:
def generate_normal_dataset(size, num_columns, mean, std_dev, output_path=None):
    """
    Generates a dataset with the specified parameters where each column
    contains integer values following a normal distribution.

    Parameters:
    - size (int): Number of rows in the dataset.
    - num_columns (int): Number of columns in the dataset.
    - mean (float): Mean of the normal distribution.
    - std_dev (float): Standard deviation of the normal distribution.
    - output_dir (str, optional): Directory to save the dataset in Parquet format. Defaults to None.

    Returns:
    - pd.DataFrame: Generated dataset as a pandas DataFrame.
    """
    # Generate random data for each column
    data = {
        f"d_{i}": np.random.normal(loc=mean, scale=std_dev, size=size).astype(int)
        for i in range(num_columns)
    }

    # Create DataFrame
    df = pd.DataFrame(data)

    # Save to output directory if specified
    if output_path:
        df.to_parquet(output_path, index=False)
        print(f"Dataset saved to {output_path}")

    return df


In [ ]:
df = generate_normal_dataset(size=100000, num_columns=3, mean=5000, std_dev=50, output_path="normal_3d_100k.parquet")

In [ ]:
df = pd.read_parquet("normal_3d_100k.parquet")

In [ ]:
df.describe()

## 6 Dimensions Dataset

In [ ]:
def generate_bounded_normal(loc, scale, size, low, high):
    result = []
    while len(result) < size:
        samples = np.random.normal(loc=loc, scale=scale, size=size)
        valid_samples = samples[(samples >= low) & (samples <= high)]
        result.extend(valid_samples.astype(int))
    return np.array(result[:size])

In [ ]:
def generate_fake_dataset(size, output_path=None):

    regions = [
        "Baden-Württemberg", "Bavaria", "Berlin", "Brandenburg", "Bremen", 
        "Hamburg", "Hesse", "Lower Saxony", "Mecklenburg-Vorpommern", 
        "North Rhine-Westphalia", "Rhineland-Palatinate", "Saarland", 
        "Saxony", "Saxony-Anhalt", "Schleswig-Holstein", "Thuringia"
    ]
    gender = ["m","f","d"]
    product_category = [chr(i) for i in range(97, 123)]

    p_product_category = np.random.zipf(1.5, len(product_category))
    p_product_category = p_product_category / np.sum(p_product_category)

    reference_dist = pd.read_parquet("reference_dist.parquet")

    ages_array = generate_bounded_normal(loc=35, scale=10, size=size, low=18, high=76)

    data = {
        "region": np.random.choice(regions, size=size, replace=True),
        "gender": np.random.choice(gender, size=size, replace=True, p=[0.48, 0.49, 03]),
        "category": np.random.choice(product_category, size=size, replace=True, p=p_product_category),
        "date": np.random.choice(reference_dist['date'], size=size, replace=True),
        "total": np.random.choice(reference_dist['total'], size=size, replace=True),
        "age": ages_array      
    }

    # Create DataFrame
    df = pd.DataFrame(data)

    # Save to output directory if specified
    if output_path:
        df.to_parquet(output_path, index=False)
        print(f"Dataset saved to {output_path}")

    return df


In [ ]:
df = generate_fake_dataset(size=200000, output_path="paper_example_200k.parquet")

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(df['age'], bins=30, kde=True, color='blue')
plt.title('Age Distribution')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Generate random ages with a normal distribution
ages_array = np.random.normal(loc=35, scale=10, size=1000).astype(int)

# Clip the ages to ensure they fall within the range of 18 to 76
ages_array = np.clip(ages_array, 18, 76)

## Kaggle

## E-Commerce Sales Dataset

In [ ]:
df = pd.read_csv("raw/amazon_sale_report.csv", low_memory=False)
df.to_parquet("raw/amazon_sale_report.parquet", index=False)

In [ ]:
df = pd.read_parquet("raw/amazon_sale_report.parquet")

In [ ]:
df.columns

In [ ]:
df_column_subset = df[['Date', 'Status',
       'Style', 'Category', 'Size',
       'Qty', 'Amount',
       'ship-state']].copy()

In [ ]:
df_column_subset['total'] = df_column_subset['Amount'].fillna(0).astype(int)

In [ ]:
df_column_subset['date'] = pd.to_datetime(df_column_subset['Date'], format='%m-%d-%y').rank(method='dense').astype(int) - 1

In [ ]:
df_column_subset[['date', 'total']].to_parquet("raw/reference_dist.parquet", index=False)

In [ ]:
df_column_subset.nunique()